1. 相邻像素在像素中可能举例较远，难以被模型识别
2. 对于大尺寸图像，全连接层容易造成模型过大，权重参数过多，消耗资源较多。

# LeNet模型

分为卷积层块和全连接层块两个部分。

## 卷积层块
卷积层块里的基本单位是**卷积层** 后接 **池化层**，前者用来识别图像里的空间模式，后者用来降低卷积层对位置的敏感性。

卷积层：5 * 5的窗口，输出接sigmoid激活函数。第一层输出通道为6，第二层输出通道数则增加到16，增加输出通道的目的是使得两层的参数尺寸类似。

池化层：窗口形状为 2 * 2，步幅为2，这样池化窗口每次覆盖的区域互不重叠。

卷积层块输出形状：（批量大小，通道，高，宽）


## 全连接层块
当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）。

全连接层块含3个全连接层，它们输出个数分别是120、84、10。

In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim

import d2lzh_pytorch as d2l

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channel, out_channels, kernel_size 
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16 * 4 * 4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
        def forward(self, img):
            feature = self.conv(img)
            # -1 表示除了将batch_size的留住，其他都平铺
            output = self.fc(feature.view(img.shape[0], -1))
            return output

In [5]:
net = LeNet()
print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [6]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)